In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sqlalchemy import create_engine
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
cnxn = create_engine('sqlite:///vn.db')

In [3]:
query = "SELECT * FROM HSX WHERE ticker = 'PVT' ORDER BY date"

In [4]:
df = pd.read_sql(query, cnxn, parse_dates=['date'])

In [5]:
df.tail()

,ticker,date,open,high,low,close,volume
1795,PVT,2020-08-26,10.9,11.4,10.90,11.20,3453880.0
1796,PVT,2020-08-27,11.2,11.3,11.10,11.20,1019160.0
1797,PVT,2020-08-28,11.4,11.5,11.30,11.35,2365230.0
1798,PVT,2020-09-01,11.7,12.0,11.65,11.90,2974670.0
1799,PVT,2020-09-03,12.0,12.1,11.75,11.85,2647810.0


In [10]:
df['mid'] = (df['high'] + df['low']) / 2

In [9]:
df['close_change'] = df['close'] - df['close'].shift(-1)

In [10]:
def change_classify(x):
    if x < 0:
        y = 0
    else:
        y = 1
    return y

In [12]:
df['close_change_classified'] = df['close_change'].apply(change_classify)

In [15]:
df = pd.get_dummies(df, columns=['close_change_classified'])

In [17]:
train_set = df.iloc[:-40,-2:]
test_set = df.iloc[-40:,-2:]

In [18]:
day_to_window = 90

In [19]:
sc = MinMaxScaler(feature_range=(0,1))
stage_train = sc.fit_transform(train_set.values.reshape(-1,1))
X_train = []
y_train = []
for i in range(day_to_window, len(stage_train)):
    X_train.append(stage_train[i-day_to_window:i, :])
    y_train.append(stage_train[i, :])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [20]:
dataset_total = pd.concat((train_set, test_set), axis = 0)
inputs = dataset_total[len(df) - len(test_set) - day_to_window:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
y_test = []
for i in range(day_to_window, day_to_window + len(test_set)):
    X_test.append(inputs[i-day_to_window:i, :])
    y_test.append(inputs[i, :])
X_test = np.array(X_test)
y_test = np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [22]:
model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(50))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

nn = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 200)#, batch_size = 32)

Epoch 1/200
108/108 [==============================] - 9s 85ms/step - loss: 0.6941 - accuracy: 0.4848 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 2/200
108/108 [==============================] - 8s 76ms/step - loss: 0.6938 - accuracy: 0.4921 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/200
108/108 [==============================] - 9s 80ms/step - loss: 0.6936 - accuracy: 0.5017 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/200
108/108 [==============================] - 9s 81ms/step - loss: 0.6935 - accuracy: 0.4843 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/200
108/108 [==============================] - 10s 89ms/step - loss: 0.6935 - accuracy: 0.5020 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/200
108/108 [==============================] - 9s 85ms/step - loss: 0.6936 - accuracy: 0.4854 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/200
108/108 [==============================] - 10s 96ms/step - loss: 0.6932 - accuracy: 0.4939 - val_loss: 0.6931 - val_accurac

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots()
ax.plot(nn.history['loss'])
ax.plot(nn.history['val_loss'], color='y')

In [13]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [14]:
real = test_set.values

In [15]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name='real',
        x=df.iloc[-40:,:]['date'],
        y=real.flatten()
    )
)

fig.add_trace(
    go.Scatter(
        name='pred',
        x=df.iloc[-40:,:]['date'],
        y=predicted_stock_price.flatten()
    )
)
fig.show()

In [40]:
mean_squared_error(real, predicted_stock_price)

5.409361499134738

In [ ]:
0.08086984669252982